<a href="https://colab.research.google.com/github/souza-dev/sistema_recomendacao/blob/main/sistema_recom_surprise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

O dataset utilizado foi obtido 
https://nijianmo.github.io/amazon/index.html#complete-data
onde foi escolhido os reviews de usuários na cateogoria CDs e Discos de Vinil.

In [1]:
!pip install surprise

In [2]:
!wget 'http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/CDs_and_Vinyl_5.json.gz' -O 'CDs_and_Vinyl_5.json.gz'

--2021-11-06 23:51:07--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/CDs_and_Vinyl_5.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 484144315 (462M) [application/octet-stream]
Saving to: ‘CDs_and_Vinyl_5.json.gz’

CDs_and_Vinyl_5.jso 100%[===================>] 461.72M  42.3MB/s    in 11s     

2021-11-06 23:51:18 (40.8 MB/s) - ‘CDs_and_Vinyl_5.json.gz’ saved [484144315/484144315]



In [3]:
!gunzip -f CDs_and_Vinyl_5.json.gz

# Recuperação dos dados e feature engineering

In [4]:
import pandas as pd
raw_data = pd.read_json('/content/CDs_and_Vinyl_5.json', lines=True)
raw_data

,reviewerID,asin,reviewerName,verified,reviewText,overall,reviewTime,summary,unixReviewTime,style,vote,image
0,A1H1DL4K669VQ9,0001393774,Judith Paladino,True,Love it!! Great seller!,5,"04 29, 2016",Five Stars,1461888000,NaN,NaN,NaN
1,A3V5XBBT7OZG5G,0001393774,gflady,True,One of my very favourite albums from one of my...,5,"02 23, 2016",One of my very favourite albums from one of my...,1456185600,NaN,NaN,NaN
2,A3SNL7UJY7GWBI,0001393774,Lady Leatherneck,True,"THank you Jesus Lord God, that brother Green's...",5,"02 11, 2016",Five Stars,1455148800,NaN,NaN,NaN
3,A3478QRKQDOPQ2,0001393774,jacki,True,I recall loving his other albums and maybe thi...,5,"11 28, 2015",forgot but I figured on some of these artists ...,1448668800,NaN,NaN,NaN
4,A23M5VTSN2C3H1,0001393774,Caliope,True,Keith Green was a pioneer in the field of Chri...,5,"12 16, 2014",and I have loved every album he did,1418688000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1443750,A30ZM19J0CNAGJ,B01HJDG28E,Reggie: a music lover,True,Night Hawk is ten outstanding new recordings o...,5,"12 31, 2016","My favorite cut is the cover of ""(Ghost) Rider...",1483142400,{'Format:': ' Audio CD'},NaN,NaN
1443751,APXJ8QC63EG38,B01HJG3VZI,Steven Schubak,True,ok,3,"07 25, 2018",Three Stars,1532476800,{'Format:': ' Audio CD'},NaN,NaN
1443752,A5HIGHKQPINEO,B01HJG3VZI,Oman,True,Great music great sound love the music lots of...,5,"07 17, 2018",The sound quality and number of hits,1531785600,{'Format:': ' Audio CD'},NaN,NaN
1443753,A1B26TNQEWHQ4K,B01HJG3VZI,David W. Barsness,True,Really good fun quality stuff,5,"06 23, 2018",5fer,1529712000,{'Format:': ' Audio CD'},NaN,NaN


In [5]:
data = raw_data[['reviewerID', 'asin', 'overall']].dropna()
data = data.sample(frac=0.3, random_state=42) #Foi usado 30% do dataset original por questões de desempenho.
data = data.reset_index(drop=True)
data.columns = ['uid', 'iid', 'rating']
data

,uid,iid,rating
0,A31OMQ0YC2Z764,B000001E0I,5
1,A196QPMC6YWUDV,B00065GHWE,5
2,A2W2PYVSXV4Z6H,B007885NOA,5
3,A1P329PIY6SJG,B00AGKH6D8,5
4,A1QZS6NVZ85F72,B000003N6H,1
...,...,...,...
433121,A1UIZW67H2318W,B00000K53Y,5
433122,A107I6YPYHLZIC,B00000AEDU,4
433123,A1IFS48OYNGOZX,B000002UDB,3
433124,APVZ7D2UB5K3N,B01F3REDMU,4


In [6]:
data.describe()

,rating
count,433126.000000
mean,4.440221
std,0.976458
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


# Uso da biblioteca Surprise para fazer as predições

In [7]:
import surprise
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split

In [8]:
reader = Reader(line_format='user item rating', rating_scale=(0.0, 5.0))

In [9]:
data_suprise = surprise.Dataset.load_from_df(data,reader)

In [10]:
trainset, testset = train_test_split(data_suprise)

In [11]:
algo = surprise.SVDpp()
algo.fit(trainset)

Funções auxiliares

In [12]:
from collections import defaultdict

#Função que retorna as 5 melhores avalicações do usuário.
def get_best_reviews(uid):
  return data.set_index('uid').loc[uid].sort_values('rating', ascending=False).reset_index(drop=True).head(5)

#Função que retorna todos os produtos avaliados pelo usuário.
def get_iid_reviewed(uid):
  return data.set_index('uid').loc[uid]['iid'].reset_index(drop=True)

#Função que retorna os produtos não avaliados pelo usuário.
def get_not_reviewed(uid):
  return data.set_index('iid').drop(get_iid_reviewed(uid), axis=0).reset_index()

#Função que retorna a predição de nota pra cada produto não avaliado pelo usuário.
def verify_predict(uid):
  dic = []
  for iid in get_not_reviewed(user_dict[0])['iid']:
    dic.append(algo.predict(uid, iid))
  return dic

#Função que retorna o dataframe ordenado por predição.
def show_recommendation(uid):
  predicts = defaultdict(list)
  for _, item, _, predict, _ in verify_predict(uid):
    predicts[item] = predict
  df = pd.DataFrame.from_dict(predicts, orient='index', columns=['Predict'])
  df = df.sort_values('Predict', ascending=False).head(5).reset_index()
  df.columns = ['iid', 'Predict']
  return df

# Utilizando as funções de predição

Como o dataset utiliza hashes para o id de usuário ao invés de número, foi criado um dicionário enumerando essas hashes e usado para mapear um número para a hash da posição correspondente. Assim a função pode ser chamada utilizando números ao invés de precisar entrar com as hashes.

In [14]:
#Dicionário auxiliar, para buscar o usuário por numero ao invés de hashes
user_dict = pd.DataFrame(pd.unique(data['uid'])).to_dict()[0]

In [15]:
from google.colab import output
def menu():
  while (True):
    output.clear()
    user = int(input(f'\nDigite um número de usuário de 0 a {len(user_dict) - 1}  para obter as recomendações: '))
    print("Produtos que o usuário avaliou bem:\n")
    best_reviews = get_best_reviews(user_dict[user])
    for iid in best_reviews['iid']:
      print('https://www.amazon.com/dp/'+iid)
    print("\n")
    print("Produtos recomendados:\n")
    recomendation = show_recommendation(user_dict[user])
    for iid in recomendation['iid']:
      print('https://www.amazon.com/dp/'+iid)
    print("\n")
    cont = input('\nDeseja consultar outro usuário? [S/N]\n ').upper()
    if cont == 'N':
      break
  

In [16]:
menu()


Digite um número de usuário de 0 a 103307  para obter as recomendações: 345
Produtos que o usuário avaliou bem:

https://www.amazon.com/dp/B000002JU3
https://www.amazon.com/dp/B00000I3ZR
https://www.amazon.com/dp/B000002BR7
https://www.amazon.com/dp/B001VFM0QK
https://www.amazon.com/dp/B00006FWXU


Produtos recomendados:

https://www.amazon.com/dp/B000002W6C
https://www.amazon.com/dp/B0000033WI
https://www.amazon.com/dp/B000000WHF
https://www.amazon.com/dp/B000000W7O
https://www.amazon.com/dp/B000002A3E



Deseja consultar outro usuário? [S/N]
 n
